# Agente simple para crear ddl a partir de tablas de un excel

## Set up modelo

In [29]:
import vertexai
import os
from dotenv import load_dotenv
from langchain_google_vertexai import ChatVertexAI

vertexai.init(
    project="gen-lang-client-0730997933",
    location="us-central1"
)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'gen-lang-client-0730997933-9e99e8511560.json'

load_dotenv()

GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

llm = ChatVertexAI(model="gemini-1.5-pro-002")

## Prompt

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# ID para guardar el historial de consultas anteriores
MEMORY_KEY = "chat_history"

# Template configurado para el prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """System Prompt:

                You are a data processing agent. Your task is to:

                1. Open an Excel file with path provided by the user.
                2. Extract all tables from the Excel sheets on a dictionary.
                3. Interpretate the dictionary and create a sql script to create these tables on database. To interpretate first get name of columns and then get the type of variable.
                4. Finally give the sql script as response.

                """,
        ),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

## Tools config

In [24]:
from langchain_core.tools import tool
from typing import List, Type, Dict
from langchain_community.document_loaders import UnstructuredExcelLoader
import pandas as pd
from langchain_core.messages import AIMessage, HumanMessage

@tool
def get_tables(file_path: str) -> Dict[str, List[Dict[str, str]]]:
    """Extrae tablas desde un archivo Excel, guarda cada tabla en un CSV y devuelve un diccionario con columnas y tipos de datos."""
    # Cargar todas las hojas del archivo Excel
    xls = pd.ExcelFile(file_path)
    
    tables_info = {}

    # Recorrer cada hoja, guardarla como CSV y extraer información sobre columnas y tipos de datos
    for sheet_name in xls.sheet_names:
        # Convertir la hoja a DataFrame
        df = xls.parse(sheet_name)
        
        # Guardar el DataFrame como CSV
        csv_file_name = f"{sheet_name}.csv"
        df.to_csv(csv_file_name, index=False)
        
        # Extraer información de las columnas y tipos de datos
        columns_info = [
            {"name": column, "dtype": str(dtype)}
            for column, dtype in df.dtypes.items()
        ]
        
        # Guardar esta información en el diccionario
        tables_info[sheet_name] = columns_info
    
    return tables_info

tools = [get_tables]

llm_with_tools = llm.bind_tools(tools)

## Agent

In [25]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentType
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],

    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [26]:
chat_history = []

agent_executor = AgentExecutor(agent=agent, tools = tools, verbose=True)

In [27]:
input = "Genera un script sql desde el archivo ubicado en './Book1.xlsx'"
result = agent_executor.stream({"input": input,"chat_history": chat_history})

In [28]:
result

<generator object AgentExecutor.stream at 0x0000020BC08EE7A0>

In [11]:
def save_script(result, file_name):
    """save script on sql file."""
    
    with open(file_name, 'w') as file:
        file.write(result)

In [12]:
result.get('output')

'```sql\nCREATE TABLE Campañas_promociones (\n  "Código de campaña" TEXT,\n  "Nombre campaña" TEXT,\n  Objetivo TEXT,\n  "Monto descuento" REAL,\n  "Tipo de descuento" TEXT,\n  "Categoría producto" TEXT,\n  "Fecha inicio" TIMESTAMP,\n  "Fecha de término" TIMESTAMP\n);\n\nCREATE TABLE campañas_2 (\n  "Código de campaña" TEXT,\n  "Nombre campaña" TEXT,\n  Objetivo TEXT,\n  "Monto descuento" TEXT,\n  "Tipo de descuento" TEXT,\n  "Categoría producto" TEXT,\n  "Fecha inicio" TIMESTAMP,\n  "Fecha de término" TIMESTAMP\n);\n```'

In [13]:
save_script(result.get('output'), 'output.sql')

# Agent 2 ddl -> html 

## Prompt

In [18]:
prompt_2 = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Eres un experto en bases de datos, lectura de script sql y programador html. Tu tarea es a traves de un script poder entender la estructura de las tablas y crear formularios en html a partir de estos",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name=MEMORY_KEY),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [21]:
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders.csv_loader import CSVLoader

@tool
def get_sql(url: str) -> str:
    """Get script from input file."""

    loader = TextLoader(url)
    
    sql = loader.load()

    sql_content = sql[0].page_content 

    return str(sql_content)

@tool
def get_data(url: str) -> str:
    """Get data example"""

    
    loader = CSVLoader(file_path='./example_data/mlb_teams_2012.csv')
    data = loader.load()

    data_content = data[0].page_content 

    return data_content

In [24]:
tools2 = [get_sql, get_data]

llm_with_tools2 = llm.bind_tools(tools2)

In [30]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent_2 = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt_2
    | llm_with_tools2
    | OpenAIToolsAgentOutputParser()
)

In [31]:
agent_executor2 = AgentExecutor(agent=agent_2, tools = tools2, verbose=True)

In [32]:
list(agent_executor2.stream({"input": "Genera un html a partir del archivo 'output.sql' ","chat_history": chat_history}))



> Entering new None chain...

Invoking: `get_sql` with `{'url': 'output.sql'}`


```sql
CREATE TABLE Campañas_promociones (
  "Código de campaña" VARCHAR(255),
  "Nombre campaña" VARCHAR(255),
  Objetivo VARCHAR(255),
  "Monto descuento" FLOAT,
  "Tipo de descuento" VARCHAR(255),
  "Categoría producto" VARCHAR(255),
  "Fecha inicio" DATETIME,
  "Fecha de término" DATETIME
);

CREATE TABLE campañas_2 (
  "Código de campaña" VARCHAR(255),
  "Nombre campaña" VARCHAR(255),
  Objetivo VARCHAR(255),
  "Monto descuento" VARCHAR(255),
  "Tipo de descuento" VARCHAR(255),
  "Categoría producto" VARCHAR(255),
  "Fecha inicio" DATETIME,
  "Fecha de término" DATETIME
);
``````html
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <title>Formulario de Campañas Promocionales</title>
</head>
<body>

<h1>Crear Campaña Promocional</h1>

<form id="campanaForm">
    <label for="codigoCampana">Código de campaña:</label><br>
    <input type="text" id="codigoCampana" name="codigoCampan

[{'actions': [ToolAgentAction(tool='get_sql', tool_input={'url': 'output.sql'}, log="\nInvoking: `get_sql` with `{'url': 'output.sql'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_sql', 'arguments': '{"url": "output.sql"}'}}, response_metadata={'safety_ratings': [], 'finish_reason': 'STOP'}, id='run-e9d74b17-df71-465c-aa47-7546402a1df0', tool_calls=[{'name': 'get_sql', 'args': {'url': 'output.sql'}, 'id': '3ca8d1c2-428b-4565-9ffe-a003a7d5e5e0', 'type': 'tool_call'}], usage_metadata={'input_tokens': 75, 'output_tokens': 7, 'total_tokens': 82}, tool_call_chunks=[{'name': 'get_sql', 'args': '{"url": "output.sql"}', 'id': '3ca8d1c2-428b-4565-9ffe-a003a7d5e5e0', 'index': None, 'type': 'tool_call_chunk'}])], tool_call_id='3ca8d1c2-428b-4565-9ffe-a003a7d5e5e0')],
  'messages': [AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'get_sql', 'arguments': '{"url": "output.sql"}'}}, response_metadata={'safety_ratings': [], 